## Sentence-level online prompty mining: XORQA

In [9]:
import copy
import re
import os, sys
import json
import glob
from collections import Counter, defaultdict

import jsonlines

from exploring_sentence_level import (
    load_model,
    mine_prompt_gt,
    segment_sentence,
    run_online_prompt_mining
)

### 0. Download dataset

```bash
cd ../scripts
bash ./download_xorqa.sh
```

### 1. Process dataset

In [10]:
XORQA_BASE_DIR = '../data/xorqa/en/tydi_xor_gp/'
xorqa_xx = {
    'train': json.load(open(os.path.join(XORQA_BASE_DIR, 'gp_squad_train_data.json'), 'r'))['data'],
      'val': json.load(open(os.path.join(XORQA_BASE_DIR, 'gp_squad_dev_data.json'), 'r'))['data'],   
}

In [11]:
xorqa_xx['train'][-1]

{'title': 'title:Vamsy_parentSection:Introduction_sectionName:Career._sectionIndex:2',
 'paragraphs': [{'context': 'He has published a short stories compilation called "Maa Pasalapudi Kathalu". Besides that compilation, Vamsy has written a wide variety of short stories since 1974 when he was 18 years old. His major works include "Mahallo kokila", "Manchupallaki", "Aa Naati Vaana Chinukulu", "Venditera Kathalu" (original scripts of "Sankarabharanam" and "Anveshana"), "Vennela Bomma", "Gokulam lo Radha", "Ravvala konda", "Sree seetarama lanchi service Rajahmundry", "Manyam rani", "Rangularatnam". He has penned around 150 short stories published in swathi weekly under title "Maa Diguwa Godavari Kathalu" For his contributions to the art of story telling with a native approach through his books he was bestowed with "Sripada Puraskhaaram" at Rajamundry on 17 April 2011.',
   'qas': [{'question': 'మా పసలపూడి కథలు పుస్తకమును ఎవరు రచించారు?',
     'answers': [{'text': 'Vamsy', 'answer_start': 1

In [12]:
def get_xorqa_answer_str(context, qas):
    context_qa_pairs = []
    for qa in qas:
        question = qa['question']
        lang = qa['lang']
        answer = qa['answers'][0]['text']
        answer_start = qa['answers'][0]['answer_start']
        context_qa_pairs.append((context, question, answer, answer_start, lang))
    return context_qa_pairs

In [13]:
xorqa_xx_dataset = defaultdict(lambda: { 'train': [], 'val': [] })

for split_name in ['train', 'val']:
    for i, item in enumerate(xorqa_xx[split_name]):
        paragraphs = item['paragraphs']
#         print('.' ,end='')
        for j, paragraph in enumerate(paragraphs):

            context = paragraph['context']
            context_qa_pairs = get_xorqa_answer_str(context=context, qas=paragraph['qas'])

            for context_qa_pair in context_qa_pairs:
                context, question, answer, answer_start, lang = context_qa_pair
                gt_sentence = mine_prompt_gt((context, question, answer, answer_start))
                qa_item = {
                     'question': question,
                     'lang': lang,
                     'context': context,
                     'segmented_context': segment_sentence(context),
                     'answer': answer,
                     'answer_start': answer_start,
                     'gt_sentence': gt_sentence,
                }
                xorqa_xx_dataset[lang][split_name].append(qa_item)

In [14]:
list(xorqa_xx_dataset.keys())

['bn', 'ja', 'ko', 'ru', 'fi', ' ar', 'te', 'ar']

In [15]:
len(xorqa_xx_dataset['ar']['val'])

485

### 2. Compute question-sentence similarity


#### 2.1 Load models

##### a) Load mUSE_small (v3) model (as a baseline)

In [4]:
muse_small_v3_model = load_model('https://tfhub.dev/google/universal-sentence-encoder-multilingual/3')


KeyboardInterrupt: 

##### b) Load teacher models

In [ ]:
XQUAD_TEACHER_DIR = '../../../CL-ReLKT_store/models/XQUAD/teacher_model/'
MLQA_TEACHER_DIR = '../../../CL-ReLKT_store/models/MLQA/teacher_model/'

In [ ]:
xquad_teacher_model = load_model(XQUAD_TEACHER_DIR)
mlqa_teacher_model = load_model(MLQA_TEACHER_DIR)

##### c) Load student models

In [ ]:
XQUAD_STUDENT_SUPPORTED_LANGS_DIR = '../../../CL-ReLKT_store/models/XQUAD/student_best_supported_languages/'
XQUAD_STUDENT_UNSUPPORTED_LANGS_DIR = '../../../CL-ReLKT_store/models/XQUAD/student_best_unsupported_languages/'

XORQA_STUDENT_SUPPORTED_LANGS_DIR = '../../../CL-ReLKT_store/models/XORQA/student_best_supported_languages/'
XORQA_STUDENT_UNSUPPORTED_LANGS_DIR = '../../../CL-ReLKT_store/models/XORQA/student_best_unsupported_languages/'

MLQA_STUDENT_SUPPORTED_LANGS_DIR = '../../../CL-ReLKT_store/models/MLQA/student_best_supported_languages/'
MLQA_STUDENT_UNSUPPORTED_LANGS_DIR = '../../../CL-ReLKT_store/models/MLQA/student_best_unsupported_languages/'

In [ ]:
xquad_student_supported_langs_model = load_model(XQUAD_STUDENT_SUPPORTED_LANGS_DIR)
xorqa_student_supported_langs_model = load_model(XORQA_STUDENT_SUPPORTED_LANGS_DIR)
mlqa_student_supported_langs_model = load_model(MLQA_STUDENT_SUPPORTED_LANGS_DIR)

xquad_student_unsupported_langs_model = load_model(XQUAD_STUDENT_UNSUPPORTED_LANGS_DIR)
xorqa_student_unsupported_langs_model = load_model(XORQA_STUDENT_UNSUPPORTED_LANGS_DIR)
mlqa_student_unsupported_langs_model = load_model(MLQA_STUDENT_UNSUPPORTED_LANGS_DIR)

In [ ]:
MODEL_MAPPING = {
  # mUSE_small
  'model-muse_small_v3': muse_small_v3_model,
  # teacher    
  'model-xquad_teacher': xquad_teacher_model,
  'model-mlqa_teacher': mlqa_teacher_model,
  # student
  'model-xquad_student_supported_langs': xquad_student_supported_langs_model,
  'model-xorqa_student_supported_langs': xorqa_student_supported_langs_model,
  'model-mlqa_student_supported_langs': mlqa_student_supported_langs_model,
  'model-xquad_student_unsupported_langs': xquad_student_unsupported_langs_model,
  'model-xorqa_student_unsupported_langs': xorqa_student_unsupported_langs_model,
  'model-mlqa_student_unsupported_langs': mlqa_student_unsupported_langs_model,
}



In [16]:
DATASET_MAPPING = {}

for lang in list(xorqa_xx_dataset.keys()):
    if len(xorqa_xx_dataset[lang]['train']) != 0:
        DATASET_MAPPING[f'dataset-xorqa_{lang.strip()}_train'] = xorqa_xx_dataset[lang]['train']
    if len(xorqa_xx_dataset[lang]['val']) != 0:
        DATASET_MAPPING[f'dataset-xorqa_{lang.strip()}_val'] = xorqa_xx_dataset[lang]['val']
print(DATASET_MAPPING.keys())

dict_keys(['dataset-xorqa_bn_train', 'dataset-xorqa_bn_val', 'dataset-xorqa_ja_train', 'dataset-xorqa_ja_val', 'dataset-xorqa_ko_train', 'dataset-xorqa_ko_val', 'dataset-xorqa_ru_train', 'dataset-xorqa_ru_val', 'dataset-xorqa_fi_train', 'dataset-xorqa_fi_val', 'dataset-xorqa_ar_train', 'dataset-xorqa_te_train', 'dataset-xorqa_te_val', 'dataset-xorqa_ar_val'])


#### 2.2 Run inference and evaluate

The following function `run_online_prompt_mining` iterates over question-answer-passage triplets $(q_i, a_i, p_i)$ and compute 
the cosine similarity scores between question $q_i$ and segmented setences $s^i_j \textrm{ where } p_i = ( s^i_0, \ldots , s^i_{|p_i| - 1} )$ , and rank each quesiton-sentence pair by similairy score. Then, it evaluate the sentence-level precision@k.  Note: There is only 1 groundtruth sentence (i.e. the sentence where the answer span is a part of). 


In [ ]:

for dataset_prefix, dataset in DATASET_MAPPING.items():
    print(f'\n\ndataset_prefix: {dataset_prefix}')
    for model_prefix, model in MODEL_MAPPING.items():
        
        print(f'\n - model_prefix: {model_prefix}')
        prefix = f'{dataset_prefix}+{model_prefix}'
        _result = run_online_prompt_mining(dataset,
                             prefix=f'{dataset_prefix}_{model_prefix}',
                             model=model)


        results[dataset_prefix][model_prefix] = _result
        print('--'*50)
    print('\n')    
    print('=='*50)
    print('\n')    
    

In [ ]:
class NumpyEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, np.ndarray):
            return obj.tolist()
        return json.JSONEncoder.default(self, obj)


#### 2.3 Write result as JSON file

In [ ]:
json.dump(results, open('./eval_results.dataset_name-xorqa.json', 'w'), ensure_ascii=False, indent=2)

### 3. Convert evaluation results to a pandas.DataFrame

In [ ]:
results = json.load(open('./eval_results.dataset_name-xorqa.json', 'r'))

In [ ]:
list(results.keys()), len(list(results.keys()))

In [ ]:
result_objs = []
for dataset_name, result_model_group in data_teacher.items():
    for model_name, (metric, raw_result) in result_model_group.items():
        top1, precision_at_k = metric
        
        result_objs.append({
            'dataset_name': dataset_name,
            'model_name': model_name,
            'precision_at_1': top1,
            'precision_at_2': precision_at_k['2'],
            'precision_at_3': precision_at_k['6'],
            'precision_at_4': precision_at_k['4'],
            'precision_at_5': precision_at_k['5'],
            'precision_at_10': precision_at_k['10'],
        })
    
df = pd.DataFrame.from_dict(result_obj)
df.to_csv('./eval_results.dataset_name-xorqa.csv')